In [ ]:
# %pip install --upgrade --quiet sentence_transformers transformers anthropic anthropic openpyxl Unidecode ftfy langchain PyPDF2

In [1]:
import re
import pandas as pd
from pathlib import Path
from unidecode import unidecode 
import ftfy
import PyPDF2
import os
import json

pd.set_option("display.width", 1000000000)
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

from tqdm import tqdm
tqdm.pandas()

import random 
random.seed(0)

import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
def clean_string(context):
    context = ftfy.fix_text(context)
    context = re.sub(r"[|;()_-]", " ", context)
    context = re.sub(r"<[^>]+>", " ", context)
    context = re.sub(r"[*]+", "x", context)
    context = re.sub(r"(?<=\d),(?=\d)", " ", context)
    context = context.replace("#", " ", -1)
    context = context.replace("‚·", " ", -1)
    context = context.replace("\n", " ", -1)
    context = context.replace("Î¼", "μ", -1)
    context = context.replace(" / ", " ", -1)
    context = context.replace("”", '"', -1)
    context = context.replace("~", " ", -1)
    context = context.replace("}", " ", -1)
    context = context.replace("{", " ", -1)
    context = context.replace("&nbsp", " ", -1)
    context = context.replace(" ", " ", -1)
    context = context.replace("&reg", "", -1)
    context = context.replace("&trade", " ", -1)
    context = context.replace("&gt", " ", -1)
    context = context.replace("&lt", " ", -1)
    context = context.replace("ã", " ", -1)
    context = context.replace("‘", " ", -1)
    context = context.replace("â", " ", -1)
    context = context.replace("€", " ", -1)
    context = context.replace("“", " ", -1)
    context = context.replace("„", " ", -1)
    context = context.replace("¢", " ", -1)
    context = context.replace("", " ", -1)
    context = context.replace("", " ", -1)
    context = context.replace("【", " ", -1)
    context = context.replace("】", " ", -1)
    context = context.replace("¨", " ", -1)
    context = context.replace("Â", " ", -1)
    context = context.replace("â", " ", -1)
    context = context.replace("–", " ", -1)
    context = context.replace("¶", " ", -1)
    context = context.replace("▶", " ", -1)
    context = context.replace("µ‰", " ", -1)
    context = context.replace("á", " ", -1)
    context = context.replace("ᵉ", " ", -1)
    context = context.replace(">", " ", -1)
    context = context.replace("<", " ", -1)
    context = context.replace("[", " ", -1)
    context = context.replace("]", " ", -1)
    context = context.replace(" w/ ", " ", -1)
    context = context.replace('"', " ", -1)
    context = context.replace("&amp", "&", -1)
    context = context.replace(".x", ".", -1)
    context = context.replace("Ü", "U", -1)
    context = context.replace("ï‚·", " ", -1)
    context = context.replace("", " ", -1)
    context = context.replace("×", "x", -1)
    context = context.replace("''", '"', -1)
    context = context.replace("—", " ", -1)
    context = context.replace("®", " ", -1)
    context = context.replace("™", " ", -1)
    context = context.replace("œ", " ", -1)
    context = context.replace("", " ", -1)
    context = context.replace("ï‚·", " ", -1)
    context = context.replace("₹", "RS ", -1)
    context = context.replace("›", " ", -1)
    context = context.replace("?", " ", -1)
    context = context.replace("ø", " ", -1)
    context = context.replace("\\t", " ", -1)
    context = context.replace("\\n", " ", -1)
    context = context.replace("\\uf0b7", " ", -1)
    context = context.replace("\\u", " ", -1)
    context = context.replace("\\n", " ", -1)
    context = re.sub(r"(?<=[A-Za-z]| ):", " ", context)
    context = re.sub(r"(?<=\d)(?: ?nos\.| ?no\.)", " ", context)
    context = re.sub(r"\.(?!\d)", " ", context)
    context = re.sub(r"(?<=\d) \.|\. (?=\d)", ".", context)
    context = re.sub(":{2,}", "", context)
    context = re.sub("\s\s+", " ", context)
    return context.strip()

In [ ]:
def get_pdf_text(pdf_doc):    
    resume_text = ""
    try:
        loader = PyPDFLoader(pdf_doc)
        documents = loader.load()
        for doc in documents:
            page_text = str(doc).replace("page_content=", "").split("metadata={'source':")[0]
            page_text = page_text.replace("\n", " ")
            resume_text += page_text
        # print(resume_text)
        resume_text = clean_string(unidecode(resume_text))
            
    except Exception as e:
        print("Exception:", e)
    return resume_text

In [ ]:
def iterate_pdf_documents(folder_path):
    folder = Path(folder_path)
    pdf_files = folder.glob("**/*.pdf")
    return pdf_files

In [ ]:
get_pdf_text('resume_folder/aaliverma91@gmail.com_resume.pdf')

In [ ]:
# current_dir = "."
# pdf_files = iterate_pdf_documents(current_dir)

# resume_summary_df = pd.DataFrame()
# # counter = 0
# for file in pdf_files: 
#     resume_summary = get_pdf_text(file)
#     resume_summary_df = pd.concat(
#         objs=[resume_summary_df, pd.DataFrame.from_dict({'file_name':[file], 'summary': [resume_summary]})],
#         ignore_index=True
#     )
#     # if counter == 10:
#     #     break
#     # counter += 1
# resume_summary_df.head(1)
# resume_summary_df.to_csv(path_or_buf='resume_summary.csv', index=False)

# Create prompt and run sungle inference on LLM

In [ ]:
import boto3

In [ ]:
os.environ['CURL_CA_BUNDLE'] = ''
from langchain import PromptTemplate

In [ ]:
bedrock = boto3.client(service_name="bedrock-runtime", region_name='us-east-1')
modelId = "mistral.mixtral-8x7b-instruct-v0:1"

accept = "application/json"
contentType = "application/json"

In [ ]:
multi_var_prompt = PromptTemplate(
    input_variables=["summary"], 
    template='''
    You are an expert whose responsibility is to extract the candiate details and generate a 350 word summary from a given resume text on the candidate education background,
    work experience as number of years followed by number of months, companies the candidate has worked in including interships, the kind of work done, professional certifications
    completed, research papers published and candidate skill set. Make sure you provide no extra information in response if relevant information is not explicitly mentioned. 
    Do not hallucinate or make assumptions about answers. Maintain consistency in our response each time. 
    
    Resume Summary: <<<{summary}>>>
'''
)


# Below is an sample of resume summary:

# I'm Arpit Jain, a Geospatial Specialist with expertise in GIS development and analysis. My background in Civil Engineering and Architecture enables me
# to design and craft impactful solutions. From 3D architectural visualizations to research protoypes , I merge creativity and precision. 
# Proficient in Unity, ArcGIS, and Python, I excel at transforming complex data into actionable insights. You can visit my port folio at ar-arpit.ml 
# to view my work and connect with me. 
# You can also get in touch with me using following - 
# Email ID: arpit66120@gmail.com   | Phone: +91(783)68 0-5636  | LinkedIn: Arpit Jain

# EDUCATION  

# ● Master of Technology in Civil Engineering with specialization in Geomatics 2021 -2023  
# Indian Institute of Technology, Roorkee, Uttarakhand

# ● Bachelor of Architecture  2014 -2020  
# School of Planning and Architecture, New Delhi.

# WORK EXPERIENCE 
# . DotNet Technologies - May 2022 to Present

#  Certifications:
# . Certification is Graphic designing

# Founded and managing a successful architectural and design consultancy, delivering comprehensive 
# services . Completed various graphic design, website design, interior design , product design and website 
# development projects  apart from architectural projects.

# Below is the sample for generated summary:

# Arpit has a strong educational background, holding a Master of Technology in Civil Engineering with a specialization in Geomatics from the 
# Indian Institute of Technology, Roorkee, completed between 2021 and 2023. Prior to this, he obtained a Bachelor of Architecture from the School of 
# Planning and Architecture, New Delhi, spanning from 2014 to 2020. In terms of work experience, Arpit has been with DotNet Technologies since May 2022,
# accumulating experience in his current role for approximately 2 years and 1 month at the time of this resume. At DotNet Technologies, 
# he founded and manages a successful architectural and design consultancy, demonstrating leadership and entrepreneurship skills. Within this role, 
# he has completed various projects encompassing graphic design, website design, interior design, product design, and website development, in addition 
# to his core architectural responsibilities. Arpit's skill set is highlighted by his proficiency in Unity, ArcGIS, and Python, enabling him to excel in
# GIS development and analysis. His expertise extends to transforming complex data into actionable insights, showcasing a blend of technical acumen and
# creativity. Additionally, he holds a certification in graphic designing, reflecting a commitment to ongoing learning and professional development in diverse domains.



In [ ]:
# summary = """
# Prasanth Coimbatore Tamil Nadu India prasanthpkp191196@gmail com 8122490546 https //www linkedin com/in/prasanth pkp 2391b4145/ SUMMARY Senior Software Engineer with 6 years of experience in developing high quality software solutions Proven track record of leading cross functional teams successfully delivering projects on time and within budget Expertise in full stack development including proficiency in React Node and AWS Cloud Applying for the positionof Senior Software Engineer at Steam A Private Limited to leverage technical skills and contribute to the company's growth and success EXPERIENCE Senior Software Engineer Steam A Private Limited October 2023 Present Coimbatore Tamil Nadu IN x Electric Vehicles Digital Advisory EV Charging Change Design and Delivery Digital Transformation Business Transformation and Energy & Utilities x Developing innovative products for mobility and smart charging bringing the future forward x Steam A professional dares to promise and the committed to delivering results Consistently And we are carefully curating a team of multipotentiality who bring holistic perspectives and solutions to the table x We leverage cutting edge design and fit for purpose technology to deliver target outcomes for our customers in energy & utilities oil and gas and mobility EV sectors We aim to harness intelligent technology and solve problems with artistic perspectives Senior Software Engineer InterviewDesk Technologies February 2022 August 2023 Chennai India x Led the development of a proprietary digital product x Utilized Front End Frameworks ReactJs Design Frameworks AntdDesign and Back End Frameworks Kotlin NodeJs x Implemented DynamoDB for efficient database management x Proficient in various AWS Services Senior Software Developer FalconOrange Technologies April 2021 February 2022 Coimbatore India x Developed and implemented web applications and bot products x Utilized Front End Frameworks ReactJs VueJs Design Frameworks AntdDesign Vuetify Bootstrap and Back End Frameworks NodeJs Python x Managed databases including PostgreSQL and MongoDB Full Stack Developer Focus 4 D Career Education Pvt Ltd February 2019 February 2021 Coimbatore India x Developed an electronic learning platform with 5+ code modules x Conducted regular code refactoring and collaborated with educators x Ensured platform scalability and incorporated gamification features x Managed third party integrations and provided comprehensive training resources Software Developer Consensus Technology June 2017 February 2019 Coimbatore India x Oversaw the entire E learning platform Dodo and contributed to application and content development x Actively enriched features in a live environment PROJECTS Charger Management Software CMS Steam A Private Limited x https //www steam a com/ x October 2023 Present x CMS is a software for Charge Point Operators CPOs to manage their Electric Vehicle charging stations across multiple locations from a single application It provides real time monitoring and optimization capabilities for charging operations allowing CPOs to grow their charging station business and scale efficiently x Engineered a robust Charger Management Software CMS platform that managed over 5 000 electric vehicle charging stations incorporating real time monitoring and automated billing features that improved system uptime by 25% x Front End Frameworks ReactJs NextJS x Design Framework Antd Design x Back End Frameworks NodeJs Java OCPP OCPI x AWS Services EC2 S3 ES Lambda CodeCommit CodePipeline CloudWatch API Gateway ' Sourcing Application Platform InterviewDesk Technologies x https // www gallophire ai/ x May 2023 September 2023 x With a reservoir of 1+ million profiles dive into a search experience that covers Keywords Skills Job Insights Experience Location Company Culture Designation and even Alma Mater x Front End Frameworks ReactJs x Design Framework Antd Design x Back End Frameworks NodeJs x Database DynamoDB x AWS Services EC2 S3 ES Lambda CodeCommit CodePipeline CloudWatch API Gateway DynamoDB Video Conference Platform InterviewDesk Technologies x https //interviewdesk ai/virtual interview platform/ x January 2023 April 2023 x Played a key role in developing a virtual interview platform with advanced AI powered tools x Managed end to end development infrastructure setup and deployment x Conducted load testing and performance tuning x Collaborated with UX/UI designers for a seamless user experience x Implemented real time chat features for a dynamic user experience x Conducted regular system backups and disaster recovery tests x Introduced version control processes for code management x Supported a global user base with 24/7 technical assistance x Resolved complex technical challenges within tight deadlines x Environment x Front End Frameworks ReactJs x Design Framework Antd Design x Back End Frameworks NodeJs x Database DynamoDB x AWS Services AWS CDK AWS SDK EC2 S3 Lambda CodeCommit CodePipeline CloudWatch API Gateway DynamoDB AWS Amplify Cognito CloudFormation App Runner Elastic Container Service Engineered a scalable video conference platform utilizing a suite of AWS services including EC2 S3 Lambda and DynamoDB which facilitated reliable real time communication for over 50 000 concurrent users during peak performance Biochemical Web Application FalconOrange Technologies x February 2022 February 2023 x Worked on multiple digital products facilitating communication between owners and users x Developed a bio chemical product x Implemented RESTful APIs for seamless data retr ieval x Conducted unit testing and test driven development TDD x Collaborated with QA teams for bug fixing and quality assurance x Supported a global user base with timely updates and support x Implemented continuous integration and continuous delivery CI/CD pipelines x Conducted comprehensive user training sessions x Ensured data security and compliance with industry standards x Front End Frameworks ReactJs TypeScript x Design Framework Antd Design x Back End Frameworks NodeJs Python FastApi x Database PostgreSQL MongoDB x AWS Services AWS CDK AWS SDK EC2 CI/CDEngineered a robust biochemical web application infrastructure using AWS CDK and AW S SDK managed multiple EC2 instances for high availability and leveraged CI/CD pipelines that ensured seamless deployment with a 99.9% uptime Cabin Bot Platform FalconOrange Technologies x April 2021 February 2022 x Developed multiple bot products enabling communication between owners and users x Managed deployment processes x Implemented real time chat functionality for enhanced user engagement x Conducted system performance optimization x Collaborated with product managers for feature planning and prioritization x Conducted user feedback sessions for product improvements x Ensured high availability of bot services x Conducted regular code refactoring for code maintainability x Supported a diverse range of industries with customized bot solutions ' ' x Front End Frameworks Bot Framework ReactJS x Back End Frameworks Node js x Database PostgreSQL x Design Framework BootstrapEngineered a responsive user interface for the Cabin Bot Platform using the Bootstrap framework leading to a 50% increase in user engagement and a more seamless experience across various devices and screen sizes Electronic Learning Platform Focus 4 D Career Education Pvt Ltd x https // www faceprep in/ x February 2019 February 2021 x Created an electronic learning platform an integrated set of interactive online services supporting education delivery and management x Worked on 5+ code modules within this project x Conducted regular code refactoring for maintainability x Collaborated with educators for platform customization x Ensured platform scalability for a growing user base x Implemented gamification features to enhance user engagement x Conducted usability testing and user feedback analysis x Managed third party integrations for enriched learning experiences x Supported instructors with comprehensive training resources x Ensured data security and compliance with education standards x Front End Frameworks ReactJs TypeScript x Design Framework Antd Design x Back End Frameworks NodeJs x Database MySQL x AWS Services AWS CDK AWS SDK EC2 CI/CD E learning and Onl ine Coding Platform Consensus Technology Coimbatore x July 2017 January 2019 x Managed the entire E learning and online coding platform Dodo x Added new features to the live environment x Part of application development and content development for courses x Conducted periodic security audits x Collaborated with customer support for issue resolution x Ensured high system uptime for uninterrupted learning x Conducted code optimization for improved platform performance x Supported instructors with course creation tools x Implemented single sign on SSO for enhanced user convenience x Introduced a comprehensive analytics dashboard for performance tracking x Front End Frameworks ReactJs JavaScript x Design Framework Antd Design x Back End Frameworks NodeJs x Database MySQL x AWS Services AWS EC2 S3 EDUCATION Bachelor of Engineering Minor in Computer Science Engineering x Muthayammal Engineering College x Namakkal India x 2018 x 6.5 Higher Secondary Education HSE Minor in Computer Science x State Board x Kallakurichi India x 2014 x 6.5 Secondary School Leaving Certificate SSLC Minor in Mathematics x State Board x Kallakurichi India x 2012 x 79 SKILLS x Front End React Vue Next js Nuxt js x Back End Node js Kotlin x Databases MySQL DynamoDB MongoDB Oracle PostgreSQL x AWS Services Extensive experience with AWS CDK AWS SDK EC2 S3 Lambda CodeCommit CodePipeline CloudWatch API Gateway DynamoDB AWS Amplify Cognito CloudFormation App Runner ECS CloudFront AWS Chime Video and AWS Chime Chat '
# """
summary = """
I'm Arpit Jain, a Geospatial Specialist with expertise in GIS development and analysis. My background in Civil Engineering and Architecture enables me
to design and craft impactful solutions. From 3D architectural visualizations to research protoypes , I merge creativity and precision. 
Proficient in Unity, ArcGIS, and Python, I excel at transforming complex data into actionable insights. You can visit my port folio at ar-arpit.ml 
to view my work and connect with me. 
You can also get in touch with me using following - 
Email ID: arpit66120@gmail.com   | Phone: +91(783)68 0-5636  | LinkedIn: Arpit Jain

EDUCATION  

● Master of Technology in Civil Engineering with specialization in Geomatics 2021 -2023  
Indian Institute of Technology, Roorkee, Uttarakhand

● Bachelor of Architecture  2014 -2020  
School of Planning and Architecture, New Delhi.

WORK EXPERIENCE 
. DotNet Technologies - May 2022 to Present

 Certifications:
. Certification is Graphic designing

Founded and managing a successful architectural and design consultancy, delivering comprehensive 
services . Completed various graphic design, website design, interior design , product design and website 
development projects  apart from architectural projects.
"""
prompt = multi_var_prompt.format(summary=summary)
body = json.dumps({
    "prompt": prompt,
    "max_tokens": 600,
    "top_p": 0.8,
    "temperature": 0.5,
})
response = bedrock.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)
response = json.loads(response.get('body').read())['outputs'][0]['text'].replace("\nSummary:\n\n", "")
response

In [ ]:
# try:
#     response = json.loads(re.sub("\s+", " ", response[0]['text'].replace("\n", "")))
# except:
#     response = re.sub("\s+", " ", response[0]['text'].replace("\n", ""))
# response['skills'] = ', '.join(response['skills'])
# response

# Get resume summary using LLM

In [ ]:
data = pd.read_csv('resume_summary.csv')
data.head(1)

In [ ]:
def get_response(summary):
    flag = True
    while(flag):
        try:
            prompt = multi_var_prompt.format(summary=summary)
            body = json.dumps({
                "prompt": prompt,
                "max_tokens": 800,
                "top_p": 0.8,
                "temperature": 0.5,
            })
            response = bedrock.invoke_model(
                body=body,
                modelId=modelId,
                accept=accept,
                contentType=contentType
            )
            flag = False
        except Exception as ex:
            x = random.randint(1, 3)
            time.sleep(x)
            
    try:
        response = json.loads(response.get('body').read())['outputs'][0]['text'].replace("\nSummary:\n\n", "")
    except:
        pass
    return response

# Test on sample of rows

In [ ]:
temp = data.sample(5)
temp['llm_summary'] = temp['summary'].progress_apply(lambda x: get_response(x))

In [ ]:
for response in temp['llm_summary']:
    print(response)

# Run of full DF

In [ ]:
counter = 0
final_df = pd.DataFrame()
while True:
    if counter > data.shape[0]:
        break
    temp = data.iloc[counter : counter + 500]
    temp['llm_summary'] = temp['summary'].progress_apply(lambda x: get_response(x))
    final_df = pd.concat([final_df, temp])
    counter += 500

In [ ]:
final_df.to_csv('llm_generated_summary.csv', index=False)

In [ ]:
data = pd.read_csv('llm_generated_summary.csv')
data.head(1)

In [ ]:
data['llm_summary'] = data['llm_summary'].str.strip()

In [ ]:
data.sample(2)

# Rankig with semantic similarity on JD requirement query

In [2]:
import re
import string
import nltk
nltk.download('punkt')
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download('stopwords')

import pandas as pd
pd.set_option("display.width", 1000000000)
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import PyPDF2
import os

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

[nltk_data] Downloading package omw-1.4 to /home/ec2-user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from sentence_transformers.cross_encoder import CrossEncoder

In [9]:
# Pre-trained cross encoder
model = CrossEncoder("BAAI/bge-reranker-base")
# model = CrossEncoder("BAAI/bge-reranker-v2-m3")

In [6]:
query = """Computer Science Engineering 2 years 1 month experience software engineering C Angular SQL Microsoft Azure Azure Data Factory CICD Azure pipelines API YAML"""

In [7]:
summary_1 = """
'Arpit Jain Master Technology Civil Engineering specialization Geomatics Indian Institute Technology Roorkee Uttarakhand 2021 2023 Bachelor Architecture School Planning Architecture New Delhi 2014 2020 working Geospatial Specialist DotNet Technologies since May 2022 proficiency Unity ArcGIS Python Arpit also completed certification Graphic Designing founded manages successful architectural design consultancy delivering comprehensive services including graphic design website design interior design product design website development projects addition architectural projects'
"""

summary_2 = """candidate Shilpa Kuriankose Bachelor Engineering Computer Science Engineering Madras Institute Technology Chennai India earned June 2020 2 years 1 month experience software engineering focus Net Framework C Angular SQL Microsoft Azure Azure Logic Apps Azure Data Factory trouble shooting worked Senior Software Engineer LTI Mindtree Chennai India March 2021 January 2024 responsible client support checking bugs weaknesses providing enhancements code bases direct client interactions deploying solutions using CICD Azure pipelines also experience Software Engineer LTI Mindtree Chennai India February 2021 August 2021 checked client codes bugs weaknesses supported RPA application made enhancements code bases Additionally experience Graduate Engineer Trainee LTI Mindtree Chennai India October 2020 February 2021 developed application called Online Examination System C front end Angular backend SQL data storage also worked projects Emed Domgas Blend Reports built NET Core Angular back ends use SQL Azure Logic Apps Blob Storage professional certifications Net Framework C Angular SQL Microsoft Azure Azure Logic Apps Azure Data Factory trouble shooting skill set includes Net Framework C Angular SQL Microsoft Azure Azure Logic Apps Azure Data Factory trouble shooting stored procedures web API YAML files
"""

In [10]:
# We want to compute the similarity between the query sentence and all resume summaries
corpus = [summary_1, summary_2]

# 1. We rank all sentences in the corpus for the query
ranks = model.rank(query, corpus)

print("Query: ", query, '\n')
for rank in ranks:
    print(f"Similarity score: {rank['score']:.2f}\n\n{corpus[rank['corpus_id']]}")
    print("~" * 50)
    
# # 2. Alternatively, you can also manually compute the score between two sentences 
# sentence_combinations = [[query, sentence] for sentence in corpus]
# scores = model.predict(sentence_combinations)

# print("scores:", scores)

Query:  Computer Science Engineering 2 years 1 month experience software engineering C Angular SQL Microsoft Azure Azure Data Factory CICD Azure pipelines API YAML 

Similarity score: 1.00

candidate Shilpa Kuriankose Bachelor Engineering Computer Science Engineering Madras Institute Technology Chennai India earned June 2020 2 years 1 month experience software engineering focus Net Framework C Angular SQL Microsoft Azure Azure Logic Apps Azure Data Factory trouble shooting worked Senior Software Engineer LTI Mindtree Chennai India March 2021 January 2024 responsible client support checking bugs weaknesses providing enhancements code bases direct client interactions deploying solutions using CICD Azure pipelines also experience Software Engineer LTI Mindtree Chennai India February 2021 August 2021 checked client codes bugs weaknesses supported RPA application made enhancements code bases Additionally experience Graduate Engineer Trainee LTI Mindtree Chennai India October 2020 February 2

In [11]:
data = pd.read_csv('single_string_llm_response_mixtral.csv', usecols=['file_name', 'llm_summary'])
data.head(1)

,file_name,llm_summary
0,resume_folder/shilpakur18@gmail.com_resume.pdf,"The candidate, Shilpa Kuriankose, has a Bachelor of Engineering in Computer Science Engineering from the Madras Institute of Technology in Chennai, India, earned in June 2020. She has 2 years and 1 month of experience in software engineering, with a focus on Net Framework, C, Angular, SQL, Microsoft Azure, Azure Logic Apps, Azure Data Factory, and trouble shooting. She has worked as a Senior Software Engineer at LTI Mindtree in Chennai, India, from March 2021 to January 2024, where she was responsible for client support, checking for bugs and weaknesses, providing enhancements to code bases, direct client interactions, and deploying solutions using CICD Azure pipelines. She also has experience as a Software Engineer at LTI Mindtree in Chennai, India, from February 2021 to August 2021, where she checked client codes for bugs and weaknesses, supported an RPA application, and made enhancements to code bases. Additionally, she has experience as a Graduate Engineer Trainee at LTI Mindtree in Chennai, India, from October 2020 to February 2021, where she developed an application called Online Examination System with C as the front end, Angular as the backend, and SQL as data storage. She has also worked on projects such as Emed Domgas and Blend Reports, which are built in NET Core with Angular back ends and use SQL Azure, Logic Apps, and Blob Storage. She has professional certifications in Net Framework, C, Angular, SQL, Microsoft Azure, Azure Logic Apps, Azure Data Factory, and trouble shooting. Her skill set includes Net Framework, C, Angular, SQL, Microsoft Azure, Azure Logic Apps, Azure Data Factory, trouble shooting, stored procedures, web API, and YAML files."


In [12]:
def apply_lemmatization(summary):
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    summary = translator.sub(' ', summary)
    word_tokens = word_tokenize(summary)
    filtered_tokens = [token for token in word_tokens if not token.lower() in stop_words]
    return " ".join(filtered_tokens)

In [13]:
data['llm_summary_processed'] = data.apply(lambda x: apply_lemmatization(x['llm_summary']), axis=1)
data.head(1)

,file_name,llm_summary,llm_summary_processed
0,resume_folder/shilpakur18@gmail.com_resume.pdf,"The candidate, Shilpa Kuriankose, has a Bachelor of Engineering in Computer Science Engineering from the Madras Institute of Technology in Chennai, India, earned in June 2020. She has 2 years and 1 month of experience in software engineering, with a focus on Net Framework, C, Angular, SQL, Microsoft Azure, Azure Logic Apps, Azure Data Factory, and trouble shooting. She has worked as a Senior Software Engineer at LTI Mindtree in Chennai, India, from March 2021 to January 2024, where she was responsible for client support, checking for bugs and weaknesses, providing enhancements to code bases, direct client interactions, and deploying solutions using CICD Azure pipelines. She also has experience as a Software Engineer at LTI Mindtree in Chennai, India, from February 2021 to August 2021, where she checked client codes for bugs and weaknesses, supported an RPA application, and made enhancements to code bases. Additionally, she has experience as a Graduate Engineer Trainee at LTI Mindtree in Chennai, India, from October 2020 to February 2021, where she developed an application called Online Examination System with C as the front end, Angular as the backend, and SQL as data storage. She has also worked on projects such as Emed Domgas and Blend Reports, which are built in NET Core with Angular back ends and use SQL Azure, Logic Apps, and Blob Storage. She has professional certifications in Net Framework, C, Angular, SQL, Microsoft Azure, Azure Logic Apps, Azure Data Factory, and trouble shooting. Her skill set includes Net Framework, C, Angular, SQL, Microsoft Azure, Azure Logic Apps, Azure Data Factory, trouble shooting, stored procedures, web API, and YAML files.",candidate Shilpa Kuriankose Bachelor Engineering Computer Science Engineering Madras Institute Technology Chennai India earned June 2020 2 years 1 month experience software engineering focus Net Framework C Angular SQL Microsoft Azure Azure Logic Apps Azure Data Factory trouble shooting worked Senior Software Engineer LTI Mindtree Chennai India March 2021 January 2024 responsible client support checking bugs weaknesses providing enhancements code bases direct client interactions deploying solutions using CICD Azure pipelines also experience Software Engineer LTI Mindtree Chennai India February 2021 August 2021 checked client codes bugs weaknesses supported RPA application made enhancements code bases Additionally experience Graduate Engineer Trainee LTI Mindtree Chennai India October 2020 February 2021 developed application called Online Examination System C front end Angular backend SQL data storage also worked projects Emed Domgas Blend Reports built NET Core Angular back ends use SQL Azure Logic Apps Blob Storage professional certifications Net Framework C Angular SQL Microsoft Azure Azure Logic Apps Azure Data Factory trouble shooting skill set includes Net Framework C Angular SQL Microsoft Azure Azure Logic Apps Azure Data Factory trouble shooting stored procedures web API YAML files


In [14]:
def get_semantic_search_ranking(query, corpus):
    sentence_combinations = [[query, sentence] for sentence in corpus]
    similarity_score_list = model.predict(sentence_combinations)
    return similarity_score_list

In [15]:
def delete_files_in_directory(directory_path):
    try:
        files = os.listdir(directory_path)
        for file in files:
            file_path = os.path.join(directory_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        return True
    except Exception as e:
        print("Exception not able to delete files.", e)
        return False

In [16]:
def make_directiory(directory_path):
    try:
        os.mkdir(dir_name)
        return True
    except Exception as e:
        print("Exception not able to Create Directory at path: ", dir_name, "\n", e)
        return False

In [17]:
def copy_files(directory_path, file_names_list):
    try:
        for ind, file in enumerate(file_names_list):
            with open(file, 'rb') as pdf_in:
                pdf_reader = PyPDF2.PdfReader(pdf_in)
                pdf_writer = PyPDF2.PdfWriter()
                for pagenum in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[pagenum]
                    pdf_writer.add_page(page)
            new_file_path = os.path.join(directory_path, f"resume_{ind + 1}.pdf")
            with open(new_file_path, 'wb') as pdf_out:
                pdf_writer.write(pdf_out)
        print("Top resumes picked and can be found at path : ", dir_name)
    except Exception as e:
        print("Exception not able to Create Directory at path: ", dir_name, "\n", e)

In [ ]:
num_top_resume = 5
while True:
    """To exit: use 'exit', 'quit', 'q', or Ctrl-D."""
    query = input('Query: ')
    if query.lower() in ["exit", "quit", "q"]:
        print('Exiting')
        break
    print("\n Please wiat while we fectch best possible results.....")
    query = apply_lemmatization(query)
    data['similarity_score'] = list(get_semantic_search_ranking(query, data['llm_summary_processed'].tolist()))
    data.sort_values(by=['similarity_score'], inplace=True, ascending=False)
    top_rows_df = data[data['similarity_score'] > 0.8]
    if not top_rows_df.empty:
        top_resume_file_names = top_rows_df['file_name'].tolist()[: num_top_resume]
        dir_name = os.path.join(os.getcwd(), "top_resume_folder")
        if os.path.exists(dir_name):
            if delete_files_in_directory(dir_name):
                copy_files(dir_name, top_resume_file_names)
        elif make_directiory(dir_name):
            copy_files(dir_name, top_resume_file_names)
    else:
        print("Sorry! No resume matches the given job description well. Refining your JD can work.")

"To exit: use 'exit', 'quit', 'q', or Ctrl-D."

Query:  ndian Institute of Technology classification Segmentation training machine learning Linear Regression Decision Tree Random Forest XGBoost data analysis  Data Structures Pandas Numpy Matplotlib Scikit learn Tensorflow PyTorch Jupyter Notebook vs Code SQL Power BI



 Please wiat while we fectch best possible results.....


In [ ]:
"""
1. 
Civil Engineer Indian Institute Technology Geospatial Specialist DotNet Technologies Unity ArcGIS Python Arpit Graphic Designing graphic design interior design product design Data Analytic

2. 
Indian Institute of Technology classification Segmentation machine learning Linear Regression Decision Tree Random Forest XGBoost data analysis 
Data Structures Pandas Numpy Matplotlib Scikit learn Tensorflow PyTorch Jupyter Notebook vs Code SQL Power BI

3. 
Ruby on Rails 4 years of experience docker containeriztion aws Azure Agile Kanban sql Postgresql Javascript ReactJS REST API CICD Couchbase OOP Engineering degree computer science

"""